In [ ]:
# !pip install openai-whisper
# !pip install -U bitsandbytes
# !pip install dotenv

In [4]:
from utils import *
import re
import os
import json
import whisper
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import BitsAndBytesConfig
from dotenv import load_dotenv
from huggingface_hub.hf_api import HfFolder

import warnings
warnings.filterwarnings("ignore")

In [5]:
load_dotenv()
hf_token = os.getenv("HF_TOKEN")
HfFolder.save_token(hf_token)

In [6]:
audio_model = whisper.load_model("tiny.en")

100%|█████████████████████████████████████| 72.1M/72.1M [00:03<00:00, 24.4MiB/s]


In [7]:
llm_model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

In [8]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="float16"
)

tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
llm_model = AutoModelForCausalLM.from_pretrained(
    llm_model_name,
    device_map="auto",
    quantization_config=bnb_config
)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [9]:
llm_pipeline = pipeline("text-generation", model=llm_model, tokenizer=tokenizer)

Device set to use cuda:0


In [10]:
audio_file = "test1.wav"
transcribed_audio = transcribe_audio(audio_model, audio_file)

In [11]:
transcribed_audio

' Book an appointment with Dr. John on Monday at 10am.'

In [12]:
response = llm_pipeline(build_prompt(transcribed_audio), max_new_tokens=100, do_sample=False)[0]["generated_text"]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [18]:
save_json_output(response)

Output saved in JSON file!
